# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

Esses valores podem ser atribuídos na função `main`, não é necessária leitura pelo teclado. Entretanto, o programa deve ser genérico, ou seja, funcionará para qualquer outro valor atribuído às 3 variáveis.

In [17]:
#include <stdio.h>

int main() {
    int n = 5; //número de empréstimos
    double s = 200.0; //parcela inicial
    double j = 1.0;
    
    double parcela = s;
    
    for (int i = 0; i < n; i++) {
        if (i != 0) {
            parcela = parcela * (1 + j / 100);
        }
        printf("%.2lf; ", parcela);
    }
}

200.00; 202.00; 204.02; 206.06; 208.12; 

## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de cada parcela X do empréstimo.

Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa em que está inserida. Considere que esta função poderia ser eventualmente reusada em outro programa e deveria ser possível fazê-lo sem que ela dependa de nada do programa em que irá se inserir.

A função deve apenas calcular uma única parcela em cada chamada, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [16]:
#include <stdio.h>

double proximaParcela(double p_anterior, double j) {
    double p_atual;
    
    p_atual = p_anterior * (1 + j / 100);
        
    return p_atual;
}

int main() {
    int n = 5; //número de empréstimos
    double s = 200.0; //parcela inicial
    double j = 1.0;
    double parcela = s;
    
    for (int i = 0; i < n; i++) {
        if (i != 0) {
            parcela = proximaParcela(parcela, j);
        }
        printf("%.2lf; ", parcela);
    }
}

200.00; 202.00; 204.02; 206.06; 208.12; 

## Exercício Parte 3 - Minimizando os Parâmetros

Na solução anterior, você deve ter usado vários parâmetros para a comunicação entre o `main` com a função `proximaParcela`, evitando usar variáveis globais. Modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar todos os valores que estejam relacionados a um empréstimo.

In [14]:
#include <stdio.h>

typedef struct emprestimo {
    int n;
    double s; //parcela inicial
    double p; //parcela atual
    double j;
} emprestimo;

emprestimo criarEmprestimo(int n, double s, double j) {
    emprestimo novo;
    
    novo.n = n;
    novo.p = s;
    novo.j = j;
    
    return novo;
}

double proximaParcela(emprestimo e) {
    double p_atual;
    
    p_atual = e.p * (1 + e.j / 100);
        
    return p_atual;
}

int main() {
    emprestimo e = criarEmprestimo(5, 200.0, 1.0);    
    
    for (int i = 0; i < e.n; i++) {
        if (i != 0) {
            e.p = proximaParcela(e);
        }
        printf("%.2lf; ", e.p);
    }
}

200.00; 202.00; 204.02; 206.06; 208.12; 

## Exercício Parte 4 - Mantendo o Estado

A função que você codificou na solução anterior provavelmente não é capaz de manter o controle do que chamaremos de **estado do empréstimo**, ou seja, dados de qual a parcela corrente ou da última parcela calculada.

Modifique a função (módulo) `proximaParcela` acima de modo que ela seja capaz de guardar o **estado do empréstimo** por sua conta, sem que o `main` precise ficar informando este estado por parâmetros (por exemplo). Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência das funções com o programa em que ela está inserida.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente. O ideal é que o main não tenha acesso ao modo como as funções vão representar o **estado do empréstimo**, para que toda a lógica do empréstimo fique dentro das funções.

In [1]:
#include <stdio.h>

typedef struct emprestimo {
    int n;
    double p_anterior; //parcela anterior
    double p_atual; //parcela atual
    double j;
} emprestimo;

emprestimo criarEmprestimo(int n, double s, double j) {
    emprestimo novo;
    
    novo.n = n;
    novo.p_anterior = s;
    novo.p_atual = s;
    novo.j = j;
    
    return novo;
}

double proximaParcela(emprestimo *e) {
    double p_atual;
    
    p_atual = e->p_anterior * (1 + e->j / 100);
        
    return p_atual;
}

double calcularEmprestimo(emprestimo *e) {
    for (int i = 0; i < e->n; i++) {
        if (i != 0) {
            e->p_anterior = e->p_atual;
            e->p_atual = proximaParcela(e);
        }
        printf("%.2lf; ", e->p_atual);
    }
}

int main() {
    emprestimo e = criarEmprestimo(5, 200.0, 1.0);
    
    calcularEmprestimo(&e);
}

200.00; 202.00; 204.02; 206.06; 208.12; 

## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada **estado de empréstimo** separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, as funções de cálculo do empréstimo ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que comporte até 100 empréstimo e, cada novo empréstimo, só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo (até o limite de 100).

O seu programa deve apresentar o exemplo acima como teste.

In [18]:
#include <stdio.h>

#define MAX 100

typedef struct emprestimo {
    int id;
    int n_total;
    int n_atual;
    double p_anterior; //parcela anterior
    double p_atual; //parcela atual
    double j;
} emprestimo;

typedef struct conjunto_e {
    int mes_atual;
    int mes_final;
    int n_e; //numero de emprestimos
    emprestimo lista_e[MAX]; //lista de emprestimos
} conjunto_e;

emprestimo criarEmprestimo(int id, int n_total, double s, double j) {
    emprestimo novo;
    
    novo.id = id;
    novo.n_total = n_total;
    novo.n_atual = 1;
    novo.p_anterior = s;
    novo.p_atual = s;
    novo.j = j;
    
    return novo;
}

int procurarMesFinal(conjunto_e *conj_e) {
    for (int o = 0; o < conj_e->n_e; o++) {
        if (conj_e->lista_e[o].n_total > conj_e->mes_final) {
            conj_e->mes_final = conj_e->lista_e[o].n_total;
        }
    }
}

void adicionarEmprestimo(conjunto_e *conj_e ,int n_total, double s, double j) {
    emprestimo novo = criarEmprestimo(conj_e->n_e+1, n_total, s, j);
    
    conj_e->lista_e[conj_e->n_e] = novo;
    
    conj_e->n_e += 1;
    procurarMesFinal(conj_e);
}

double proximaParcela(emprestimo *e) {
    double p_atual;
       
    p_atual = e->p_anterior * (1 + e->j / 100);
        
    return p_atual;
}

double calcularEmprestimo(emprestimo *e) {
    if (e->n_atual != 1) {
        e->p_anterior = e->p_atual;
        e->p_atual = proximaParcela(e);
    }
    printf("Emprestimo %d: parcela %d eh %.2lf\n", e->id, e->n_atual, e->p_atual);
    e->n_atual += 1;
}

void processarEmprestimos(conjunto_e conj_e) {
    for (int y = 1; y <= conj_e.mes_final; y++) {
        for (int k = 0; k < conj_e.n_e; k++) {
            if (y <= conj_e.lista_e[k].n_total) {
                calcularEmprestimo(&conj_e.lista_e[k]);
            }
        }
    }
}


int main() {
    conjunto_e conj_e;
    conj_e.n_e = 0;
    conj_e.mes_atual = 1;
    conj_e.mes_final = 1;
    
    adicionarEmprestimo(&conj_e, 5, 200.0, 1.0);
    adicionarEmprestimo(&conj_e, 7, 500.0, 2.0);
    processarEmprestimos(conj_e);
}

Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
